In [253]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sig 
import pandas as pd

plt.style.use( ['science', 'no-latex'])

In [254]:
FRAME_LENGTH = 1440
HOP_LENGTH = 360

def logattacktime(signal, HEIGHT, dev=0):
   
    
    RMS=librosa.feature.rms(signal, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0] # find RMS of signal
    
    frames = range(len(RMS)) # frames in RMS
    t= librosa.frames_to_time(frames , hop_length=HOP_LENGTH) 
    
    peaksample, _ = sig.find_peaks(RMS, width=1, distance=1, height=HEIGHT) #sample of peak of RMS signal
    peaksample = peaksample[0] - dev
    
    peak_ninety = 0.9 * RMS[peaksample] #Amplitude value of 90% of peak
    peak_twenty = 0.2 * RMS[peaksample]
    
    #threshold for 90%
    k = 1
    while peak_ninety <= RMS[peaksample - k]:
        k += 1
    
    j = k - 1
    
    #linear interpolate time for peak ninety
    # x = x1 +(y-y1)*(x2-x1)/(y2-y1)
    # x = x{p-k} + (y{p_ninety} - y{p-k})(x{p-j}-x{p-k})/(y{p-j} - y{p-k})
    
    t_ninety = t[peaksample-k] + (peak_ninety - RMS[peaksample-k])*((t[peaksample-j]-t[peaksample-k])/(RMS[peaksample-j]-RMS[peaksample-k]))
    
    #20% threshold
    n = 1
    while peak_twenty <= RMS[peaksample - n]:
        n += 1
    
    m = n - 1
    
    t_twenty = t[peaksample-n] + (peak_twenty - RMS[peaksample-n])*((t[peaksample-m]-t[peaksample-n])/(RMS[peaksample-m]-RMS[peaksample-n]))
    
    lat = np.log10(t_ninety - t_twenty)
    return [lat, t_ninety, t_twenty, peaksample, RMS[peaksample], peak_ninety, peak_twenty]


def plot_LATRMS(signal, height, title="Log Attack Time", xlim=0.3, ylim=1, dev=0):
    plt.figure(figsize=(8,5))
    plt.rcParams.update({'font.size': 20})
    
    RMS = librosa.feature.rms(signal, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)[0]
    
    librosa.display.waveplot(signal, alpha=0.5, x_axis='s')
    plt.plot(t, RMS, color="r", marker='')
    plt.plot(t[logattacktime(signal, height, dev)[3]],logattacktime(signal, height, dev)[4], 'o', color='orange')
    plt.plot(logattacktime(signal,height, dev)[1], logattacktime(signal,height, dev)[5], "x", color = 'g')
    plt.plot(logattacktime(signal,height, dev)[2], logattacktime(signal,height, dev)[6], "x", color = 'g')
    
    #plt.xlabel('Time(s)')
    plt.ylabel('Normalised Amplitude')
    #plt.title(title)
    plt.ylim(-0, ylim)
    plt.xlim(0.0,xlim)

In [255]:
# Generating file names for loading audio into dataframe

# file name parameters
Amps = ['Dry_', 'Vox_', 'JFET_' ]
Chords = ['E_Chord', 'D_Chord']
Gain = ['Hi_Gain_', "Med_Gain_", 'Lo_Gain_']
Inten = ['_Hi', "_Med", "_Lo"]

# E Chords
tone_combos_Ech = []
for amp in range(len(Amps)):
    for i in range(len(Inten)):
        for c in range(len(Chords)):
            if amp == 0:
                tone_combos_Ech.append(Amps[amp]+Chords[c]+Inten[i]) 
            else:
                for g in range(len(Gain)):
                    tone_combos_Ech.append(Amps[amp]+Gain[g]+Chords[c]+Inten[i])

df = pd.DataFrame(tone_combos_Ech, columns=['Note'])
df.set_index('Note', drop=True, inplace=True)

df.head(5)

""
Note
Dry_E_Chord_Hi
Dry_D_Chord_Hi
Dry_E_Chord_Med
Dry_D_Chord_Med
Dry_E_Chord_Lo


E CHord Load

In [77]:
Audio_arr = []

for e in range(len(df.index)):
    sound_name = df.index[e]
    y, sr = librosa.load('Renders2/Notes/'+ sound_name + '.wav', sr=48000,duration=5)
    y = librosa.util.normalize(y)
    Audio_arr.append(np.array(y))

df['Audio'] = Audio_arr 
    

In [78]:
# Find log attack time stat for each file and append to df

data = []
for i in range(len(df.index)):
    sound_name = df.index[i]
    y, sr = librosa.load('Renders2/Notes/'+ sound_name + '.wav', sr=48000,duration=5)
    y = librosa.util.normalize(y)
    data.append(logattacktime(y,0.1,)[0])

df['LogAttackTime'] = data

Init D chord load

In [79]:
df

,LogAttackTime,Audio
Note,,
Dry_E_Chord_Hi,-1.137826,"[0.0, 3.5762787e-07, 4.7683716e-07, 9.536743e-..."
Dry_D_Chord_Hi,-1.013256,"[0.0, -2.5327338e-06, -6.05042e-06, -9.84952e-..."
Dry_E_Chord_Med,-0.830553,"[0.0, -4.4602634e-06, -8.920527e-06, -1.226572..."
Dry_D_Chord_Med,-0.618102,"[0.0, -9.552005e-06, -1.910401e-05, -2.8936958..."
Dry_E_Chord_Lo,-0.885962,"[0.0, 1.6328463e-06, 4.898539e-06, 8.9806545e-..."
Dry_D_Chord_Lo,-0.679752,"[0.0, 2.6773001e-05, 5.27347e-05, 8.113031e-05..."
Vox_Hi_Gain_E_Chord_Hi,-1.201078,"[0.0, -9.1458384e-07, -2.2864594e-06, -3.20104..."
Vox_Med_Gain_E_Chord_Hi,-1.209100,"[0.0, -2.1852631e-07, -1.3111579e-06, -2.84084..."
Vox_Lo_Gain_E_Chord_Hi,-1.219077,"[0.0, 4.1178077e-07, 9.608218e-07, 1.3726026e-..."


In [10]:
# Original D Chord load
# Audio_arr = []
# for e in range(len(df_DChord['Dchord'])):
#     sound_name = df_DChord['Dchord'][e]
#     y, sr = librosa.load('Renders/DChord/'+ sound_name + '.wav', sr=48000,duration=5)
#     y = librosa.util.normalize(y)
#     Audio_arr.append(np.array(y))

    
# df_DChord['Audio'] = Audio_arr 

# dataD = []
# for i in range(len(df_DChord['Audio'])):
#     sound_name = df_DChord['Dchord'][i]
#     y, sr = librosa.load('Renders/DChord/'+ sound_name + '.wav', sr=48000,duration=5)
#     y = librosa.util.normalize(y)
#     dataD.append(logattacktime(y,0.1,)[0])

# df_DChord['LogAttackTime'] = dataD

In [256]:
#Play soume audio
frames = range(len(y))
t= librosa.frames_to_time(frames , hop_length=HOP_LENGTH, sr=48000)
sound_name

'JFET_Lo_Gain_D_Chord_Lo'

In [88]:
df.loc[df.index.str.contains('Chord_Hi') & df.index.str.contains('Lo_Gain')]

,LogAttackTime,Audio
Note,,
Vox_Lo_Gain_E_Chord_Hi,-1.219077,"[0.0, 4.1178077e-07, 9.608218e-07, 1.3726026e-..."
Vox_Lo_Gain_D_Chord_Hi,-1.025595,"[0.0, 5.321068e-07, 1.0642136e-06, 1.5963204e-..."
JFET_Lo_Gain_E_Chord_Hi,-1.227848,"[0.0030509634, 0.00021566905, -0.0015302647, -..."
JFET_Lo_Gain_D_Chord_Hi,-0.894473,"[-0.00019979572, 6.703889e-05, 0.00013969188, ..."


In [124]:
df_HiG = pd.DataFrame(df.loc[df.index.str.contains('Hi_Gain') | df.index.str.contains('Dry')]  )

df_HiG.drop(columns=['Audio'], inplace=True)

df_HiG

,LogAttackTime
Note,
Dry_E_Chord_Hi,-1.137826
Dry_D_Chord_Hi,-1.013256
Dry_E_Chord_Med,-0.830553
Dry_D_Chord_Med,-0.618102
Dry_E_Chord_Lo,-0.885962
Dry_D_Chord_Lo,-0.679752
Vox_Hi_Gain_E_Chord_Hi,-1.201078
Vox_Hi_Gain_D_Chord_Hi,-1.195068
Vox_Hi_Gain_E_Chord_Med,-1.292729


In [121]:
# df_new = pd.DataFrame({'Hi': df_hiG[df_hiG['Echord'].str.contains('d_Hi')]['LogAttackTime'],
#          'Med': df_hiG[df_hiG['Echord'].str.contains('d_Med')]['LogAttackTime'],
#          'Lo': df_hiG[df_hiG['Echord'].str.contains('d_Lo')]['LogAttackTime']})

# df_new

# # df of on High Gain High Intensity 
# tmp = df_hiG[df_hiG['Echord'].str.contains('_Hi') | df_hiG['Echord'].str.contains('Dry')]




In [188]:
def sort_In(df):
    #This sorts the dataframe of all the notes into their respective amplifier in order of cord and intensity
    
    # Creating series of each amps lats for each intensity, lo , med ,hi
    Dry_lat = df[df.index.str.contains('Dry')]['LogAttackTime']
    JFET_lat = df[df.index.str.contains('JFET')]['LogAttackTime']
    Vox_lat = df[df.index.str.contains('Vox')]['LogAttackTime']
    
    
    #data = {'Dry':Dry_lat,'Vox':Vox_lat,'Jfet':JFET_lat, }
    #f_sorted = pd.DataFrame(data=data)
    
    return (Dry_lat, JFET_lat, Vox_lat)
    

In [246]:
Dry = sort_In(df)[0]
Hi = Dry[Dry.index.str.contains('d_Hi')]
Med = Dry[Dry.index.str.contains('d_Med')]
Lo = Dry[Dry.index.str.contains('d_Lo')]

# def reindexChord(series):
#     for i in series.index:
#         if 'E_Chord_Hi' in series.index[i]: 
           
#         elif 'D_Chord' in series.index[i]:
#             series.index[i].rename('Dchord',inplace=True)
#     return series
df_test = pd.DataFrame({'Lo':Lo,'Med':Med,'Hi':Hi})

In [251]:
df_test

,Lo,Med,Hi
Note,,,
Dry_D_Chord_Hi,NaN,NaN,-1.013256
Dry_D_Chord_Lo,-0.679752,NaN,NaN
Dry_D_Chord_Med,NaN,-0.618102,NaN
Dry_E_Chord_Hi,NaN,NaN,-1.137826
Dry_E_Chord_Lo,-0.885962,NaN,NaN
Dry_E_Chord_Med,NaN,-0.830553,NaN


Note
Dry_E_Chord_Lo   -0.885962
Dry_D_Chord_Lo   -0.679752
Name: LogAttackTime, dtype: float64

In [257]:
#Graphing EChord High Gain Log Attack Times as a Test

x = ['Lo', 'Med', 'Hi']

plt.figure(figsize=(10,10))

plt.scatter(x, Dry_lat, label=' Dry') 
plt.scatter(x, JFET_lat, label='JFET') 
plt.scatter(x, Vox_lat, label='Vox') 
plt.legend(fontsize=15, frameon=True)

plt.show()

NameError: name 'Dry_lat' is not defined

<Figure size 720x720 with 0 Axes>